In [34]:
from utils import Utils
from tqdm import tqdm 
import re

In [2]:
utils = Utils("../.env")

In [4]:
vals = utils.get_full()

Scanned 530 items
Scanned 1059 items
Scanned 1594 items
Scanned 2128 items
Scanned 2660 items
Scanned 3200 items
Scanned 3731 items
Scanned 4263 items
Scanned 4797 items
Scanned 5326 items
Scanned 5855 items
Scanned 6401 items
Scanned 6931 items
Scanned 7471 items
Scanned 8003 items
Scanned 8533 items
Scanned 9067 items
Scanned 9603 items
Scanned 10143 items
Scanned 10676 items
Scanned 11209 items
Scanned 11748 items
Scanned 12288 items
Scanned 12833 items
Scanned 13366 items
Scanned 13900 items
Scanned 14440 items
Scanned 14974 items
Scanned 15512 items
Scanned 16046 items
Scanned 16591 items
Scanned 17125 items
Scanned 17659 items
Scanned 18197 items
Scanned 18735 items
Scanned 19161 items


In [21]:
vendors = list(vals.keys())
vendors[0:10]

['RED LASER TECHNOLOGY, INC',
 'RED RIVER HARLEY DAVIDSON',
 'RAYMOND STORAGE CONCEPTS INC',
 'MAUDSLEY, ERIC',
 'ALAMO RENT-A-CAR',
 'COMPLETE DRIVES INC',
 'BOB PRICE CHEVROLET',
 'DANE COUNTY CHRYSLER JEEP/DODGE RAM',
 'BURR02',
 'GFL Environmental Inc.']

In [42]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from geonamescache import GeonamesCache

In [66]:
nltk.download('punkt')
nltk.download('stopwords')
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english"))
custom_stopwords = {
    "llc", "inc", "corp", "company", "corporation", "group", 
    "limited", "technologies", "solutions", "systems", 
    "enterprises", "international", "global", "services",
    "industries", "manufacturing", "partners", "holdings"
}
us_states = ["alabama", "alaska", "arizona", "arkansas", "california", "colorado", "connecticut", "delaware", "florida", "georgia", "hawaii", "idaho", "illinois", "indiana", "iowa", "kansas", "kentucky", "louisiana", "maine", "maryland", "massachusetts", "michigan", "minnesota", "mississippi", "missouri", "montana", "nebraska", "nevada", "new hampshire", "new jersey", "new mexico", "new york", "north carolina", "north dakota", "ohio", "oklahoma", "oregon", "pennsylvania", "rhode island", "south carolina", "south dakota", "tennessee", "texas", "utah", "vermont", "virginia", "washington", "west virginia", "wisconsin", "wyoming"]
us_states_abbreviations = ["al", "ak", "az", "ar", "ca", "co", "ct", "de", "fl", "ga", "hi", "id", "il", "in", "ia", "ks", "ky", "la", "me", "md", "ma", "mi", "mn", "ms", "mo", "mt", "ne", "nv", "nh", "nj", "nm", "ny", "nc", "nd", "oh", "ok", "or", "pa", "ri", "sc", "sd", "tn", "tx", "ut", "vt", "va", "wa", "wv", "wi", "wy"]

stop_words.update(custom_stopwords)
stop_words.update(us_states)
stop_words.update(us_states_abbreviations)
gc = GeonamesCache()
place_names = set()
for place_data in gc.get_cities().values():
    place_names.add(place_data["name"].lower())

[nltk_data] Downloading package punkt to /home/siyer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/siyer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
def remove_city(word):
    tokens = word.split(" ")
    filtered_tokens = []
    i = 0
    while i < len(tokens):
        if i + 2 < len(tokens):
            three_word_sequence = " ".join(tokens[i:i+3])
            if three_word_sequence in place_names:
                i += 3
                continue

        if i + 1 < len(tokens):
            two_word_sequence = " ".join(tokens[i:i+2])
            if two_word_sequence in place_names:
                i += 2
                continue

        if tokens[i] in place_names:
            i += 1
            continue

        filtered_tokens.append(tokens[i])
        i += 1

    return " ".join(filtered_tokens)

In [78]:
def token_and_stem(item):
    input_string = item.lower()
    no_long_numbers = re.sub(r'\b\d{3,}\b', '', input_string)    
    alphanum = re.sub(r'[^a-zA-Z0-9]', ' ', no_long_numbers)
    no_places = remove_city(alphanum)
    tokens = word_tokenize(no_places)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    return ' '.join(stemmed_tokens)

In [79]:
def function_tester(func, data):
    parsed_items = {}

    for item in tqdm(data, desc="Processing", unit="item"):
        new = func(item)
        if new not in parsed_items:
            parsed_items[new] = []
        parsed_items[new].append(item)
    
    print("Original Length: ", len(data))
    print("New Length: ", len(parsed_items))
    return parsed_items

In [80]:
processed_dict = function_tester(token_and_stem, vendors)

Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19161/19161 [00:02<00:00, 7019.33item/s]

Original Length:  19161
New Length:  15111


In [81]:
sorted_keys = sorted(processed_dict.keys())

for key in sorted_keys:
    print(f"{key}: {processed_dict[key]}")



: ['MARION TOYOTA', 'WALKER TOYOTA OF ALEXANDRIA', 'REPUBLIC SERVICES #800', 'ASHLAND HONDA TOYOTA', 'BENTLEY CLEVELAND', 'OAKVILLE HONDA', 'VISTA 5072, LLC', 'GARIBALDI DE MEXICO', 'REPUBLIC SERVICES #208', 'HONDA OF FAIRFIELD', '22107 MAU LLC', 'BENTLEY NASHVILLE', 'Willoughby Industries, Inc.', 'GREER INDUSTRIES INC.', 'Allen Honda', 'SO. CA EDISON COMPANY', 'BENTLEY PARSIPPANY', 'Bryan College Station Toyota', 'PALM BEACH TIRE INC', 'HONDA OF KIRKLAND', 'University Of Houston', 'NICE SYSTEMS INC', 'Addison Group', 'TEXAS TOYOTA', 'TEO TECHNOLOGIES, INC', 'BENTLEY BEVERLY HILLS', 'BENTLEY HONOLULU', '**ROSETTA DIXON', 'BENTLEY PALM BEACH', 'University of Illinois at Chicago', 'REPUBLIC SERVICES #922', 'DESOTO HONDA', 'REPUBLIC', 'TOYOTA OF GLADSTONE', 'TOYOTA OF LAREDO', 'University of Arizona', 'BENTLEY MANHATTAN', 'FOREST TECHNOLOGIES INC', 'ENTERPRISE FLEET', 'THE YOUNG INDUSTRIES INC', 'Ontario Systems', 'AT&T CORP', 'HONDA OF CLEVELAND', 'LA PISA', 'Elizabeth Allen', 'Crystal S

In [83]:
from fuzzywuzzy import fuzz
potential_matches = {}
for word1 in sorted_keys:
    potential_matches[word1] = []
    for word2 in unique_stems:
        similarity_score = fuzz.ratio(word1, word2)
        if similarity_score >= 80 and word1 != word2:
            potential_matches[word1].append(word2)
potential_matches

/home/siyer/.pyenv/versions/3.10.10/envs/treya/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


{'': [],
 '07 17 kia': [],
 '1 30 fuccillo kia': [],
 '1 30 fuccillo kia wesley chape': [],
 '1 30 fucillo kia': [],
 '1 automot': ['automot'],
 '1 druri reimburs': [],
 '1 got junk': [],
 '1 pallet': ['pallet', 'pallet,'],
 '1 radiat ac': [],
 '1 self storag': [],
 '1 travel': ['travel'],
 '1 unifi collabor ltd': [],
 '10': ['120',
  '310',
  '140',
  '107',
  '104',
  '106',
  'a10',
  '103',
  '102',
  '109',
  '105',
  '101',
  '010',
  '210'],
 '123netinc': [],
 '1st choic collis': [],
 '1st choic reimburs': [],
 '1st kia': [],
 '1tech resourc': [],
 '1worldsync': ['1worldsync,'],
 '2 brother lawn care': [],
 '2 pillsburi street': [],
 '24 7 custom canada': [],
 '27 virtual consult': [],
 '28th st ago 28': [],
 '2checkout com': ['2checkout.com'],
 '2co com ultraedit': ['2co.com*ultraedit.co'],
 '2ring america': [],
 '3 g trim': [],
 '3 sigma': ['sigma'],
 '3 state harley davidson': [],
 '36 precis drive': [],
 '3ccomm': ['comm', '3ccomm,'],
 '3gtms': [],
 '3h communic': ['communic